# HW L4

ENGN 2912V

Seokkee Min

In [1]:
import jax
import numpy as np
import jax.numpy as jnp
import jax.lax as jal
from jax import jit, grad, vmap
import time
import sympy as sym
import matplotlib.pyplot as plt

##### Hardware:

CPU: Intel i7-8750H @ 2.20GHz 

GPU: NVIDIA GeForce RTX 1060

In [2]:
# Define F(x) Function

#jax function
def fx(x):
    sig = np.sqrt(0.2)
    miu = 0
    y = (1/(sig*jnp.sqrt(2*np.pi)))*jal.exp((-1/2)*(x-miu)**2/(sig)**2)    
    return y

# symbolic function and derivatives
def fx_(x_):
    sig = np.sqrt(0.2)
    miu = 0
    y_ = (1/(sig*sym.sqrt(2*np.pi)))*sym.exp((-1/2)*(x_-miu)**2/(sig)**2)    
    dy = y_.diff(x_)
    ddy = y_.diff(x_, x_)
    dddy = y_.diff(x_, x_, x_)
    return y_, dy, ddy, dddy
    
x_ = sym.symbols('x_')
y_, dy, ddy, dddy = fx_(x_)

In [3]:
# Define input variable X
x = np.linspace(-5, 5, 10000, dtype=np.float64)
x = jnp.array(x)

2023-04-13 22:21:13.635856: E external/xla/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NOT_INITIALIZED: initialization error
No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [4]:
# x10 Iteration Timing Function 
def looper(x, cpu, gpu):
    t1 = np.zeros(10)
    t2 = np.zeros(10)
    t3 = np.zeros(10)
    tc = np.zeros(10)
    tg = np.zeros(10)
    for i in range(10):
        t1[i] = time.perf_counter()
        vmap(cpu)(x)
        t2[i] = time.perf_counter()
        vmap(gpu)(x)
        t3[i] = time.perf_counter()
    
    tc = t2-t1
    tg = t3-t2
    
    return tc, tg

### 0-Order - f(x)

In [5]:
fx_gpu = jit(fx, backend = 'gpu')
fx_cpu = jit(fx, backend = 'cpu')
fx_g = vmap(fx_gpu)(x)
fx_c = vmap(fx_cpu)(x)

RuntimeError: Unknown backend: 'gpu' requested, but no platforms that are instances of gpu are present. Platforms present are: interpreter,cpu

In [ ]:
tc0, tg0 = looper(x, fx_cpu, fx_gpu)

In [ ]:
# Number of Interations
trial = np.linspace(1,10,10)

In [ ]:
plt.plot(trial, tc0, 'r-o', label='CPU')
plt.plot(trial, tg0, 'b-o', label='GPU')
plt.legend()
plt.xlabel('# of trials')
plt.ylabel('Computation Time [s]')
plt.title('F(x) Computation Time')
plt.show()
print('CPU Computation Time: mean = {} s, st.dev = {} s'.format(np.mean(tc0), np.std(tc0)))
print('GPU Computation Time: mean = {} s, st.dev = {} s'.format(np.mean(tg0), np.std(tg0)))
# print('CPU Computation Time After 1st Iteration: mean = {} s, st.dev = {} s'.format(np.mean(tc0[1:-1]), np.std(tc0[1:-1])))
# print('GPU Computation Time After 1st Iteration: mean = {} s, st.dev = {} s'.format(np.mean(tg0[1:-1]), np.std(tg0[1:-1])))

For the 0-order calculation, the GPU calculations generally appear to be slightly faster than the CPU. Both sets appear to be iterating faster overall as the number of iterations increase

In [ ]:
fa = [y_.subs({x_: point}) for point in x]

In [ ]:
plt.plot(x, fx_g, 'r-', label = "JAX")
plt.plot(x, fa, 'k-.', label = "SymPy")
plt.title('F(x)')
plt.legend()
plt.show()

Jax and SymPy representation of F(x) align.

### First Derivative - f'(x)

In [ ]:
dfx_gpu = jit(grad(fx), backend = 'gpu')
dfx_cpu = jit(grad(fx), backend = 'cpu')
df_g = vmap(dfx_gpu)(x)
df_c = vmap(dfx_cpu)(x)

In [ ]:
tc1, tg1 = looper(x, dfx_cpu, dfx_gpu)

In [ ]:
plt.plot(trial, tc1, 'r-o', label='CPU')
plt.plot(trial, tg1, 'b-o', label='GPU')
plt.legend()
plt.xlabel('# of trials')
plt.ylabel('Computation Time [s]')
plt.title('1st Derivative Computation Time')
plt.show()
print('CPU Computation Time: mean = {} s, st.dev = {} s'.format(np.mean(tc1), np.std(tc1)))
print('GPU Computation Time: mean = {} s, st.dev = {} s'.format(np.mean(tg1), np.std(tg1)))
# print('CPU Computation Time After 1st Iteration: mean = {} s, st.dev = {} s'.format(np.mean(tc1[1:-1]), np.std(tc1[1:-1])))
# print('GPU Computation Time After 1st Iteration: mean = {} s, st.dev = {} s'.format(np.mean(tg1[1:-1]), np.std(tg1[1:-1])))

For the first derivative of f(x), the CPU and GPU calculation times are approximately equal, with the GPU being slightly faster overall. The calculation time decreases overall as the number of trials increase.

In [ ]:
dfa = [dy.subs({x_: point}) for point in x]

In [ ]:
plt.plot(x, df_g, 'r-', label = "JAX")
plt.plot(x, dfa, 'k-.', label = "SymPy")
plt.title('1st Derivative')
plt.legend()
plt.show()

Jax and SymPy representation of the 1st derivative of F(x) are in alignment.

### 2nd Derivative - f''(x)

In [ ]:
#jax
ddfx_gpu = jit(grad(grad(fx)), backend = 'gpu')
ddfx_cpu = jit(grad(grad(fx)), backend = 'cpu')

ddf_g = vmap(ddfx_gpu)(x)
ddf_c = vmap(ddfx_cpu)(x)

In [ ]:
# %timeit ddf_g.block_until_ready

In [ ]:
#calculation times
tc2, tg2 = looper(x, ddfx_cpu, ddfx_gpu)

In [ ]:
#sympy substitution
dy2_n = [ddy.subs({x_: point}) for point in x]

In [ ]:
plt.plot(trial, tc2, 'r-o', label='CPU')
plt.plot(trial, tg2, 'b-o', label='GPU')
plt.legend()
plt.xlabel('# of trials')
plt.ylabel('Computation Time [s]')
plt.title('2nd Derivative Computation Time')
plt.show()
print('CPU Computation Time: mean = {} s, st.dev = {} s'.format(np.mean(tc2), np.std(tc2)))
print('GPU Computation Time: mean = {} s, st.dev = {} s'.format(np.mean(tg2), np.std(tg2)))
# print('CPU Computation Time After 1st Iteration: mean = {} s, st.dev = {} s'.format(np.mean(tc2[1:-1]), np.std(tc2[1:-1])))
# print('GPU Computation Time After 1st Iteration: mean = {} s, st.dev = {} s'.format(np.mean(tg2[1:-1]), np.std(tg2[1:-1])))

GPU calculations appear to be slightly faster than CPU calculations again, overall. Similar to the 1st derivative calculations, there appears to be a slight decrease in computation time as iterations increase.

In [ ]:
plt.plot(x, ddf_g, 'r-', label='GPU')
plt.plot(x, dy2_n, 'k-.', label='SymPy')
plt.legend()
plt.xlabel('x')
plt.ylabel('d2y/dx2')
plt.title('2nd Derivative')
plt.show()

Jax (GPU) and SymPy representations of the 2nd derivative of F(x) are in alignment.

### 3rd Derivative - f'''(x)

In [ ]:
#jax
dddfx_gpu = jit(grad(grad(grad(fx))), backend = 'gpu')
dddfx_cpu = jit(grad(grad(grad(fx))), backend = 'cpu')

dddf_g = vmap(dddfx_gpu)(x)
dddf_c = vmap(dddfx_cpu)(x)

In [ ]:
# %timeit dddf_g.block_until_ready

In [ ]:
#calculation times
tc3, tg3 = looper(x, dddfx_cpu, dddfx_gpu)

In [ ]:
#sympy substitution
dy3_n = [dddy.subs({x_: point}) for point in x]

In [ ]:
plt.plot(trial, tc3, 'r-o', label='CPU')
plt.plot(trial, tg3, 'b-o', label='GPU')
plt.legend()
plt.xlabel('# of trials')
plt.ylabel('Computation Time [s]')
plt.title('3rd Derivative Computation Time')
plt.show()
print('CPU Computation Time: mean = {} s, st.dev = {} s'.format(np.mean(tc3), np.std(tc3)))
print('GPU Computation Time: mean = {} s, st.dev = {} s'.format(np.mean(tg3), np.std(tg3)))
# print('CPU Computation Time After 1st Iteration: mean = {} s, st.dev = {} s'.format(np.mean(tc2[1:-1]), np.std(tc2[1:-1])))
# print('GPU Computation Time After 1st Iteration: mean = {} s, st.dev = {} s'.format(np.mean(tg2[1:-1]), np.std(tg2[1:-1])))

GPU calculations appear to be faster than the CPU calculations, yet again. Calculation time appears to decrease as the number of iterations increase.

In [ ]:
plt.plot(x, dddf_g, 'r-', label='GPU')
plt.plot(x, dy3_n, 'k-.', label='SymPy')
plt.legend()
plt.xlabel('x')
plt.ylabel('d^3y/dx^3')
plt.title('3rd Derivative')
plt.show()

Jax (GPU) and SymPy representations of the 3rd derivative of F(x) are in alignment. 

### Runtime vs Derivative Order

In [ ]:
# CPU
plt.plot(trial, tc0, 'k-o', label='0-order')
plt.plot(trial, tc1, 'r-o', label='1st-order')
plt.plot(trial, tc2, 'b-o', label='2nd-order')
plt.plot(trial, tc3, 'm-o', label='3rd-order')

plt.legend()
plt.xlabel('# of trials')
plt.ylabel('Computation Time [s]')
plt.title('CPU Computation Time per Derivative Order')
plt.show()

print('CPU 0-Order Computation Time: mean = {} s, st.dev = {} s'.format(np.mean(tc0), np.std(tc0)))
print('CPU 1st-Order Computation Time: mean = {} s, st.dev = {} s'.format(np.mean(tc1), np.std(tc1)))
print('CPU 2nd-Order Computation Time: mean = {} s, st.dev = {} s'.format(np.mean(tc2), np.std(tc2)))
print('CPU 3rd-Order Computation Time: mean = {} s, st.dev = {} s'.format(np.mean(tc3), np.std(tc3)))

Looking at only the CPU calculation times, generally the time required appears to decrease as the number of iterations increase. This trend holds true for all of the derivative orders. There appears to be no particular relationship between calculation time and the order of the derivative.

In [ ]:
# GPU
plt.plot(trial, tg0, 'k-o', label='0-order')
plt.plot(trial, tg1, 'r-o', label='1st-order')
plt.plot(trial, tg2, 'b-o', label='2nd-order')
plt.plot(trial, tg3, 'm-o', label='3rd-order')

plt.legend()
plt.xlabel('# of trials')
plt.ylabel('Computation Time [s]')
plt.title('GPU Computation Time per Derivative Order')
plt.show()

print('GPU 0-Order Computation Time: mean = {} s, st.dev = {} s'.format(np.mean(tg0), np.std(tg0)))
print('GPU 1st-Order Computation Time: mean = {} s, st.dev = {} s'.format(np.mean(tg1), np.std(tg1)))
print('GPU 2nd-Order Computation Time: mean = {} s, st.dev = {} s'.format(np.mean(tg2), np.std(tg2)))
print('GPU 3rd-Order Computation Time: mean = {} s, st.dev = {} s'.format(np.mean(tg3), np.std(tg3)))

For the GPU calculation times, generally the time required appears to decrease as the number of iterations increase. This trend holds true for all of the derivative orders. There appears to be no particular relationship between calculation time and the order of the derivative.

### time.performance_counter() vs %timeit
Checking to see if the times results above are confounded with the time.performance_counter() api

In [ ]:
# CPU calculation times, in sequence of increasing order of derivative
%timeit -n 1 -r 10 fx_c.block_until_ready
%timeit -n 1 -r 10 df_c.block_until_ready
%timeit -n 1 -r 10 ddf_c.block_until_ready
%timeit -n 1 -r 10 dddf_c.block_until_ready

In [ ]:
# GPU calculation times, in sequence of increasing order of derivative
%timeit -n 1 -r 10 fx_g.block_until_ready
%timeit -n 1 -r 10 df_g.block_until_ready
%timeit -n 1 -r 10 ddf_g.block_until_ready
%timeit -n 1 -r 10 dddf_g.block_until_ready

The %timeit api returns similar magnitude of calculation times for both CPU and GPU, on the order of a few hundred nanoseconds. This is ~10^4 magnitude faster than the previous method, of taking the difference between time.performance_counter() api time outputs before and after the calculation. 

As I wasn't able to get a number format (e.g. double, float, etc) output from the %timeit api, I wasn't able to replicate the calculation time vs iteration plots for it, and visually show whether or not the previous trend of the calculation time decrease per iteration also holds true for this api. However, as I increase the number of iterations, the average time does decrease so it stands to reason that there is a similar dependence on the number of iterations:

In [ ]:
# number of iterations increase per line
%timeit -n 1 -r 1 dddf_g.block_until_ready
%timeit -n 1 -r 2 dddf_g.block_until_ready
%timeit -n 1 -r 5 dddf_g.block_until_ready
%timeit -n 1 -r 10 dddf_g.block_until_ready